In [ ]:
from flask import Flask
from flask import request, Response
import json
import requests
import pickle
import pandas as pd
import numpy as np

##################################################################
app = Flask(__name__)
##################################################################
def read_from_pickle(pickle_fname):
    with open('pickles/' + pickle_fname+'.pickle', 'rb') as handle:
        return pickle.load(handle)
    
def get_df(batch_json):
    dict = batch_json
    try:
        df = pd.DataFrame(dict, index=[0], columns=dict.keys())
    except Exception as e: 
        print(e)
    return df
def get_value_slice(n, bins, labels):
    for i,x in enumerate(bins):
        try:
            if n >= bins[i] and n < bins[i+1]:
                return labels[i]
        except Exception as e:
            # print(str(e), '-----', n, labels)
            return labels[-1]
        
def set_category(df, init_col, target_col, bins, labels):
    df[target_col] = df.apply(lambda x: get_value_slice(x[init_col], bins, labels), axis=1)
##################################################################
clf = read_from_pickle('clf_TO_SVM')
features = read_from_pickle('features')

feature_variables = ['Age', 'Gender', 'MaritalStatus', 'Education', 'EducationField', 'WorkLifeBalance', 
                     'Department', 'EnvironmentSatisfaction', 'JobSatisfaction', 'OverTime', 'JobLevel',
                     'JobInvolvement', 'PerformanceRating', 'TrainingTimesLastYear', 'BusinessTravel',
                     'MonthlyIncome', 'PercentSalaryHike', 'DistanceFromHome', 'NumCompaniesWorked', 
                     'TotalWorkingYears', 'YearsWithCurrManager'] 
columns_cat = ["Gender", "Age_Group", "MaritalStatus", "Education", "EducationField", "WorkLifeBalance", 
               "Department", "EnvironmentSatisfaction", "JobSatisfaction", 
               "OverTime", "JobLevel", "JobInvolvement",  "PerformanceRating", 
               'YearsWithCurrManager', 'NumCompaniesWorked', 
               "TrainingTimesLastYear", "BusinessTravel"]

data_df = pd.read_excel('data/IBM-Dataset-HR-Employee-Attrition.xlsx')
data_df = data_df[feature_variables]
##################################################################
@app.errorhandler(405)
def handle_405(e):
    return '{"Error":"Not allowed method !"}', 405
##################################################################
@app.errorhandler(404)
def handle_404(e):
    return '{"Error":"The requested URL was not found on the server. !"}', 404
##################################################################
##################################################################
@app.route("/api/predict_attrition/", methods=['POST'])
@app.route("/api/predict_attrition", methods=['POST'])
def api_get_attrition():
    emp_batch = '{"employee_info": {}}'
    data = request.data
    if data:
        try:
            emp_batch = json.loads(data)
            try:
                input_df = get_df(emp_batch["employee_info"])[feature_variables]
                
                df = input_df.append(data_df).reset_index()
                df = df.drop('index', axis=1)
                
                df['Age_Group'] = ''
                
                set_category(df,'Age', 'Age_Group', [0, 25, 35, 45, 55, 150],['17-25','25-35','35-45','45-55','55>'])
                
                for col in columns_cat:
                    df_ = pd.get_dummies(df[col], prefix=col)
                    df = df.drop(col, axis=1)
                    df = pd.concat([df, df_], axis=1)
                    
                df = df.drop('Age', axis=1)
                df = df[features]
                
                sample = df[df.index == 0]
                
                prediction = int(clf.predict(sample)[0])
                prediction_class = 'Yes' if prediction == 1 else "No"
                prediction_likelihood = clf.predict_proba(sample)[0][1]
            except Exception as e:
                return '{"Error":"Input data passed with bad attribute names or values !"}' + str(e), 400
            result = '{"attrition_classification": "' + prediction_class + '", {"attrition_likelihood": "' + str(prediction_likelihood) + '"}'
            return result
        except:
            return '{"Error":"Input data not passed in the right JSON format !"}', 400
    else:
        return '{"Error": "No data passed !"}', 400
##################################################################
##################################################################
if __name__ == "__main__":
    app.run(threaded=True)
#################################################################


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Apr/2019 12:23:37] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 12:23:48] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 12:23:55] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 12:24:03] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 12:24:23] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 12:24:40] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 12:24:58] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 12:25:03] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 12:25:13] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 12:25:28] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 12:25:34] "POST /api/predict_attrition/ HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2019 